In [203]:
from Bio.PDB import *
import glob
CaMKII = glob.glob("input.loop3/CaMKII_loop3_*.pdb")
holoCaM = glob.glob("input.loop3/holoCaM_loop3_*.pdb")
Ng = glob.glob("input.loop3/Ng-*_loop3_*.pdb")
loop3Size = 176

files = glob.glob("output.loop3/*.txt")
charge = dict()
for chgFile in files:
    prot = chgFile.split('/')[1].split('.')[1]
    with open(chgFile,"r") as file1:
        for line in file1:
            line = line.strip().split()
            charge[(prot,line[0])] = line[1]

output = open("distData.txt", 'w+')


def count(gen):
    return sum(1 for _ in gen.get_atoms())

parser = PDBParser()
structure = parser.get_structure('loop3', 'input.loop3/CaMKII_loop3_1.0.pdb')
if count(structure) != loop3Size:
    print("Error in atom number!\n")
    raise SystemExit(1)

for i in structure.get_atoms():
    if i.parent.get_resname() == ' CA':
        calAtom = i


In [204]:
f=list(structure.get_residues())[0]

In [205]:
map(lambda x: print(x), f.get_atoms())

In [206]:
fatoms = [ x for x in f.get_atoms()]

In [207]:
fatoms[175]

IndexError: list index out of range

In [208]:
len(fatoms)

6

In [209]:
for i in structure.get_atoms():
    print(i)

<Atom CA>
<Atom C>
<Atom O>
<Atom H01>
<Atom H02>
<Atom H03>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom OD1>
<Atom OD2>
<Atom H>
<Atom HA>
<Atom HB1>
<Atom HB2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD>
<Atom CE>
<Atom NZ>
<Atom H>
<Atom HA>
<Atom HB1>
<Atom HB2>
<Atom HG1>
<Atom HG2>
<Atom HD1>
<Atom HD2>
<Atom HE1>
<Atom HE2>
<Atom HZ1>
<Atom HZ2>
<Atom HZ3>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom OD1>
<Atom OD2>
<Atom H>
<Atom HA>
<Atom HB1>
<Atom HB2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom H>
<Atom HA1>
<Atom HA2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom ND2>
<Atom OD1>
<Atom H>
<Atom HA>
<Atom HB1>
<Atom HB2>
<Atom HD21>
<Atom HD22>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom H>
<Atom HA1>
<Atom HA2>
<Atom N>
<Atom CA>
<Atom C>
<Atom O>
<Atom CB>
<Atom CG>
<Atom CD1>
<Atom CD2>
<Atom CE1>
<Atom CE2>
<Atom CZ>
<Atom OH>
<Atom H>
<Atom HA>
<Atom HB1>
<Atom HB2>
<Atom HD1>
<Atom HD2>
<Atom HE

In [210]:
for i in structure.get_atoms():
    if i.parent.get_resname() == ' CA':
        calAtom = i

In [211]:
calAtom.serial_number

173

In [212]:
for j in structure.get_atoms():
    j.parent.get_atoms()

In [213]:
j

<Atom HW2>

In [214]:
k = j.parent

In [215]:
calAtom.element

'C'

In [216]:
np.linalg.norm(j.coord - calAtom.coord)

3.3365405

In [217]:
import numpy as np

In [218]:
element = ['H','C','N','O','S']

In [219]:
def cutoff(r_ij, r_cut):
    if r_ij > r_cut:
        return 0
    else:
        return 0.5*(math.cos(math.pi*r_ij/r_cut)+1)

In [220]:
import math

In [221]:
# feature vector has 80 elements:
# Matrix Rad 0-54: radial (0-10: H; 11-21: C; 22-32: O; 33-43: N; 44-54: S)
# Matrix Ang 55-79: angular (H-H H-C H-O H-N H-S; C-H C-C C-O C-N C-S;)

import numpy as np
rad_shape = (5,11)
ang_shape = (5,5)

rad = np.zeros(rad_shape, dtype=np.float64)
ang = np.zeros(ang_shape, dtype=np.float64)


In [222]:
ele = dict()
ele['C']=[]
ele['N']=[]
ele['O']=[]
ele['H']=[]
ele['S']=[]

for i in atoms:
    if not i is calAtom:
        ele[i.element].append(i.serial_number-1)    

In [223]:
def Rad(atom, r_min, r_max, r_inc):
    tau = 0.0001
    at = atoms[atom]
    for element in ele:
        atomlist = ele[element]
        for r_cut in np.arange(r_min,r_max+0.5*r_inc,r_inc):
            #print("rcut is %f" %r_cut)
            g_rad = 0
            for i in atomlist:
                a = atoms[i]
                r_ij = np.linalg.norm(a.coord - at.coord)
                #print("rij is %f" %r_ij)
                g_rad += np.exp(-tau*r_ij*r_ij)*cutoff(r_ij, r_cut)
            fea_vec.append(g_rad)        
    return 0

In [224]:
fea_vec = []
Rad(172, 1.0, 6.0, 0.5)
fea_vec

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.047307210128145,
 1.1967975981125274,
 1.5066243488597375,
 2.004955285442836,
 2.8411140121393226,
 4.033719454229774,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0009897846583811192,
 0.09339170505723056,
 0.39191070138236994,
 0.8629700274066399,
 0.0,
 0.0,
 0.0,
 0.00039646571481836014,
 0.27490274428395056,
 0.9963190278332895,
 1.8101630630505012,
 2.599736335974363,
 3.3203209099274806,
 3.981303365423585,
 4.621246827341096,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0200044769546624,
 0.19312552221281395,
 0.6740455357328887,
 1.5077658408839807,
 2.618024169674673,
 4.072176863568633,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [225]:
def Ang(atom, r_cut):
    tau = 0.0001
    n = 0.5
    at = atoms[atom]
    for element in ele:
        atomlist = ele[element]
        for element2 in ele:
            atomlist2 = ele[element2]
            g_ang = 0
            for k in atomlist:
                for j in atomlist2:
                    ak = atoms[k]
                    aj = atoms[j]
                    r_ij = np.linalg.norm(at.coord - aj.coord)
                    r_ik = np.linalg.norm(at.coord - ak.coord)
                    r_kj = np.linalg.norm(ak.coord - ak.coord)

                    cos_ikj = np.inner((at.coord - aj.coord), (at.coord - ak.coord))/(r_ij*r_ik)
                    #print("cosine angle is %f " % cos_ikj)
                    f1 = np.power(round(1-cos_ikj, 6),n)
                    f2 = np.exp(-tau*(r_ij*r_ij+r_ik*r_ik+r_kj*r_kj))
                    f3 = cutoff(r_ij, r_cut)*cutoff(r_ik, r_cut)*cutoff(r_kj, r_cut)
                    #print("%f %f %f" %(f1, f2, f3))
                    g_ang += f1*f2*f3
            fea_vec.append(g_ang)

In [233]:
Ang(172, 3.0)

/home/pengzhi/Programs/miniconda3/envs/h2o/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in float_scalars


In [227]:
fea_vec

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.047307210128145,
 1.1967975981125274,
 1.5066243488597375,
 2.004955285442836,
 2.8411140121393226,
 4.033719454229774,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0009897846583811192,
 0.09339170505723056,
 0.39191070138236994,
 0.8629700274066399,
 0.0,
 0.0,
 0.0,
 0.00039646571481836014,
 0.27490274428395056,
 0.9963190278332895,
 1.8101630630505012,
 2.599736335974363,
 3.3203209099274806,
 3.981303365423585,
 4.621246827341096,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0200044769546624,
 0.19312552221281395,
 0.6740455357328887,
 1.5077658408839807,
 2.618024169674673,
 4.072176863568633,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 nan,
 nan,
 nan,
 nan,
 0,
 nan,
 0.0,
 0.0,
 0.0,
 0,
 nan,
 0.0,
 0.06639951705840369,
 0.0,
 0,
 nan,
 0.0,
 0.0,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0]

In [228]:
len(fea_vec)

80

In [229]:
a=[1]

In [230]:
a+a

[1, 1]

In [231]:
for i in ele:
    for j in np.arange(1.0,6.5,0.5):
        print(str(i)+str(j))

C1.0
C1.5
C2.0
C2.5
C3.0
C3.5
C4.0
C4.5
C5.0
C5.5
C6.0
N1.0
N1.5
N2.0
N2.5
N3.0
N3.5
N4.0
N4.5
N5.0
N5.5
N6.0
O1.0
O1.5
O2.0
O2.5
O3.0
O3.5
O4.0
O4.5
O5.0
O5.5
O6.0
H1.0
H1.5
H2.0
H2.5
H3.0
H3.5
H4.0
H4.5
H5.0
H5.5
H6.0
S1.0
S1.5
S2.0
S2.5
S3.0
S3.5
S4.0
S4.5
S5.0
S5.5
S6.0


In [232]:
for i in ele:
    for j in ele:
        print(i+j)

CC
CN
CO
CH
CS
NC
NN
NO
NH
NS
OC
ON
OO
OH
OS
HC
HN
HO
HH
HS
SC
SN
SO
SH
SS
